## Reading in a short story as text sample into Python

In [71]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print((f"Total number of characters: {len(raw_text)}"))
print(raw_text[:99])

Total number of characters: 20480
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Using the `re.split` command to split a text on whitespace characters:

In [72]:
import re
text = "Hello, world. This is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


Splitting the expressin above on whitespaces, commas, and periods:

In [73]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


Removing redundant whitespace characters from the result above:

In [74]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', 'is', 'a', 'test', '.']


Whether to keep or remove the whitespace characters depends on the application. 

Enabling the code above the ability to handle other types of punctuation:

In [75]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Applying the basic tokenizer above on Edith Wharton's etire short story:

In [76]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


Printing the first 30 tokens:

In [77]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Converting tokens into token IDs

Creating a list of all unique tokens sorted alphabatically:

In [78]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

1130


Creating the "vocabulary":

In [79]:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


The dictionary contains individual tokens associated with unique integer labels. 

Implementing a simple text tokenizer:

In [80]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab #A
        self.int_to_str = {i:s for s, i in vocab.items()} #B

    def encode(self, text): #C
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids): #D
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #E
        return text

Instantiating a new tokenizer object from the `SimpleTokenizerV1` class and using it to tokenize a passage from Edith Wharton's short story:

In [81]:
tokenizer = SimpleTokenizerV1(vocab)

text = """
"It's the last he painted, you know," 
Mrs. Gisburn said with pardonable pride.
"""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


Turning the token IDs back to text using the `decode` method:

In [82]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


If we use this simple tokenizer on a text that contains words that were not used in *The Verdict* story, it will throw an error, because of new words that will not be in its dictionary.  How to handle unknown words will be discussed in the next session.   

## Adding special context tokens

These special context tokens can include `<|unk|>` and `<|endoftext|>` for marking unknown words and document boundaries.

Modifying the vocabulary to include the two special tokens mentioned above:

In [83]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


Printing the last  5 entries in the updated vocablulary:

In [84]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


Adjusting the tokenizer code to include these two special tokens:

In [85]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()]\')',r'\1', text)
        return text
        

Trying out the modified tokenizer:

In [86]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the place."

text = " <|endoftext|> ".join((text1, text2))

print(text)


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the place.


In [87]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 773, 7]

In [88]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|> , do you like tea ? <|endoftext|> In the sunlit terraces of the place .'

## BytePair encoding

- This is used in GPT-2.  
- Tokens can be smaller than a word.  

In [90]:
import tiktoken
# print(f"tiktoken version: {version("tiktoken")}")

Instantiating the BPE tokenizer from tiktoken:

In [91]:
tokenizer = tiktoken.get_encoding("gpt2")

Using the BPE tokenizer:

In [96]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [97]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


## Data sampling with a sliding window

Since LLMs are trained to generate one word at a time, it is necessary to prepare the training data in such a way that the next word in a sequence represents the target to predict.

In [98]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5146


- For each chunk, we want the inputs and targets  
- To create the sliding window, the targets are the inputs shifted by one position to the right.

In [100]:
enc_sample = enc_text[50:]

context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- One by one, the prediction would look like as follows:

In [101]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [103]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- Next-word prediction will be taken up in a later chapter.  
- For now, a simple data loader that iterates over the input dataset and rrturns the inputs and targets shifted by one will suffice.

In [105]:
import torch
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.8.0


Creating the dataset and dataloader that extracts chunks from the input text dataset:

In [124]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
        

In [125]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader


Testing the dataloader with a batch size of 1for an LLM with a context size of 4:

In [126]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [128]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1,
    max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [129]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


Creating batched outputs with increased `stride` since overlaps between the batches coulde lead to increased overfitting:

In [132]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4,
    stride=4, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs: \n", inputs)
print("\nTargets: \n", targets)

Inputs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets: 
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## Creating token embeddings

- data are almost ready for an LLM.  
- but tokens are better embedded in a continuous vector representation using an embedding layer.  
- these embedding layers are usually part of the LLM itself and are updated during model training.

Example of what the input looks like after tokenization:

In [133]:
input_ids = torch.tensor([2, 3, 5, 1])

Creating a simple vocabulary 6 words long, to create embeddings for the example tensor above:

In [134]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

This will result in a 6x3 weight matrix:

In [135]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- the embedding layer is simply a more efficient way of implementing one-hot encoding.  
- this matrix can be seen as a neural network layer that can be optimized via backpropagation.

Converting a token given an `id` of `3` into a 3-dimensional vector:

In [136]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]],
       grad_fn=<EmbeddingBackward0>)


The result here is the 4th row in the `embedding_layer` weight matrix.

Embedding all four `input_id`s:

In [137]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]],
       grad_fn=<EmbeddingBackward0>)


The embedding layer can be seen as a lookup operation!

## Encoding word positions

- The embedding layer converts IDs into identical vector representations regardless of where they are located in the input sequence  
- The BytePair encoder has a vocabulary of 50,257.  

Example showing how to encode the input tokens into a 256-dimensional vector representation:

In [138]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

Sampling data from the dataloader and embedding the tokens in each batch into a 256-dimensional vector with a batch size of 8 and token length of 4 results in a 8x4x256 tensor:

In [139]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [141]:
# Calculating the dimensions
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735,
          -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991,
          -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737,
          -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,
           0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584,
          -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,
           0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,
           1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,
           0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,
           0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,
           0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231,
          -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,
           0.8473]],

Since GPT-2 uses absolute position embeddings, it is necessary to create another embedding layer using similar approach:

In [142]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,
          1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498,
         -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,
          0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,
          0.4827]], requires_grad=True)


Calculating the shape of the position embeddings using similar approach:

In [144]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


Creating the input embeddints used in an LLM by adding the token embeddings and the positional embeddings:

In [145]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,
           0.8900],
         [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493,
          -1.6473],
         [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,
           0.8051],
         [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,
           0.8310]],

        [[ 3.2835,  1.1749, -1.4150,  ..., -0.3281,  2.4332,
           0.6924],
         [-0.2199, -0.9114, -0.1750,  ...,  1.5337, -0.1998,
           0.1462],
         [ 1.5197, -1.4240,  0.4391,  ...,  1.0494, -1.4318,
           2.3057],
         [ 0.2893,  0.8346, -0.1884,  ...,  1.9602,  0.8709,
           0.8796]],

        [[ 0.9662,  0.0952, -0.4640,  ..., -1.0320,  1.6290,
           1.7771],
         [ 2.4468, -0.2154,  1.4984,  ...,  1.8766,  0.5595,
          -0.1423],
         [-0.3856, -2.5393,  1.1556,  ...,  3.6157,  1.3267,
           0.4944],
         [-0.2487, -0.5275,  2.0009,  ...,  0.2930,  0.5977,
           1.3300]],

- In this initial phase of input processing, the input text is segmented into separate tokens.  
- These segmented tokens are transformed into token IDs based on a predefined vocabulary.
